# Geo-Safe Monitor PH: Landslide Risk POC

This notebook mirrors the prototype workflow for monitoring vegetation change over Baungon, Bukidnon using Sentinel-2 imagery. It incorporates the fixes highlighted in the earlier code review, including reliable dependency handling, historical data availability checks, and explicit map rendering.


In [1]:
import importlib.util
import subprocess
import sys

DEPENDENCIES = {
    'ee': 'earthengine-api',
    'geemap': 'geemap',
    'pandas': 'pandas',
    'matplotlib': 'matplotlib',
}

for module_name, package_name in DEPENDENCIES.items():
    if importlib.util.find_spec(module_name) is None:
        print(f'Installing {package_name}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])


In [2]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

try:
    ee.Initialize()
    print('Google Earth Engine initialized.')
except Exception:
    print('Authenticating with Google Earth Engine...')
    ee.Authenticate()
    ee.Initialize()
    print('Authentication complete.')


Google Earth Engine initialized.


## Configuration and Helper Functions

Define the ROI, time windows, and utility functions for cloud masking, NDVI computation, and data sourcing with a fallback for historical imagery.


In [3]:
def mask_s2_clouds(image):
    """Mask clouds in a Sentinel-2 image using the QA60 band."""
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)


def add_ndvi(image):
    """Add an NDVI band to a Sentinel-2 image."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def add_ndwi(image):
    """Add an NDWI band to a Sentinel-2 image."""
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)


def get_sentinel_collection(start_date, end_date, region, max_cloud=20):
    sr_id = 'COPERNICUS/S2_SR_HARMONIZED'
    sr_collection = (
        ee.ImageCollection(sr_id)
        .filterDate(start_date, end_date)
        .filterBounds(region)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud))
    )

    sr_count = sr_collection.size().getInfo() or 0
    if sr_count > 0:
        return sr_collection, sr_id

    toa_id = 'COPERNICUS/S2'
    toa_collection = (
        ee.ImageCollection(toa_id)
        .filterDate(start_date, end_date)
        .filterBounds(region)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud))
    )

    toa_count = toa_collection.size().getInfo() or 0
    if toa_count == 0:
        raise RuntimeError(
            f'No Sentinel-2 imagery available for {start_date} to {end_date} in the selected ROI.'
        )

    print(f'Fallback to {toa_id} for {start_date} - {end_date}.')
    return toa_collection, toa_id


In [4]:
roi = ee.Geometry.Rectangle([124.60, 8.30, 124.70, 8.40])

start_date_recent = '2025-01-01'
end_date_recent = '2025-10-01'
start_date_historical = '2020-01-01'
end_date_historical = '2020-12-31'

VEGETATION_LOSS_THRESHOLD = -0.15
VEGETATION_NDVI_THRESHOLD = 0.2
WATER_NDWI_THRESHOLD = 0.1

recent_collection, recent_dataset = get_sentinel_collection(start_date_recent, end_date_recent, roi)
historical_collection, historical_dataset = get_sentinel_collection(start_date_historical, end_date_historical, roi)

recent_image = add_ndwi(add_ndvi(recent_collection.map(mask_s2_clouds).median()))
historical_image = add_ndwi(add_ndvi(historical_collection.map(mask_s2_clouds).median()))

recent_ndvi = recent_image.select('NDVI')
historical_ndvi = historical_image.select('NDVI')

recent_ndwi = recent_image.select('NDWI')
historical_ndwi = historical_image.select('NDWI')

vegetation_mask_recent = recent_ndvi.gt(VEGETATION_NDVI_THRESHOLD)
vegetation_mask_historical = historical_ndvi.gt(VEGETATION_NDVI_THRESHOLD)
vegetation_mask_both = vegetation_mask_recent.And(vegetation_mask_historical)

water_mask_recent = recent_ndwi.gt(WATER_NDWI_THRESHOLD)
water_mask_historical = historical_ndwi.gt(WATER_NDWI_THRESHOLD)
water_mask_combined = water_mask_recent.Or(water_mask_historical)

historical_ndvi_display = historical_ndvi.where(water_mask_historical, -0.1).updateMask(vegetation_mask_historical.Or(water_mask_historical))
recent_ndvi_display = recent_ndvi.where(water_mask_recent, -0.1).updateMask(vegetation_mask_recent.Or(water_mask_recent))

ndvi_difference = recent_ndvi.subtract(historical_ndvi)
ndvi_difference_masked = ndvi_difference.updateMask(vegetation_mask_both.And(water_mask_combined.Not()))

dem = ee.Image('USGS/SRTMGL1_003').clip(roi)
slope = ee.Algorithms.Terrain(dem).select('slope')

veg_loss = historical_ndvi.subtract(recent_ndvi).rename('VEG_LOSS')
veg_loss_norm = veg_loss.max(0).unitScale(0, 0.4).clamp(0, 1)

moisture_norm = recent_ndwi.max(0).unitScale(0, 0.6).clamp(0, 1)
slope_norm = slope.unitScale(5, 45).clamp(0, 1)

risk_score = (
    veg_loss_norm.multiply(0.45)
    .add(moisture_norm.multiply(0.35))
    .add(slope_norm.multiply(0.20))
).rename('RISK_SCORE')

risk_score_masked = risk_score.updateMask(vegetation_mask_both.And(water_mask_combined.Not()))
percentile_dict = risk_score_masked.reduceRegion(
    reducer=ee.Reducer.percentile([85, 90]),
    geometry=roi,
    scale=30,
    maxPixels=1e9,
    bestEffort=True,
    tileScale=2,
)

p90 = ee.Number(percentile_dict.get('RISK_SCORE_p90'))
p85 = ee.Number(percentile_dict.get('RISK_SCORE_p85'))
risk_threshold = ee.Number(ee.Algorithms.If(p90, p90, p85))
risk_threshold = ee.Number(ee.Algorithms.If(risk_threshold, risk_threshold, 0.45))

risk_hotspots = risk_score_masked.gte(risk_threshold).selfMask().rename('RISK_HOTSPOTS')

try:
    risk_threshold_value = risk_threshold.getInfo()
    print(f'High-risk threshold set at: {risk_threshold_value:.2f}')
except Exception as exc:
    print(f'Unable to retrieve risk threshold: {exc}')

print(f'Using {recent_dataset} for the recent period and {historical_dataset} for the historical baseline.')

try:
    mgb_landslide = ee.Image('projects/sat-io/open-datasets/MGB/MGB_Landslide_Susceptibility').clip(roi)
    hazard_viz = {'min': 1, 'max': 3, 'palette': ['#1a9850', '#fee08b', '#d73027']}
    hazard_labels = {1: 'Low', 2: 'Moderate', 3: 'High'}
except Exception as exc:
    mgb_landslide = None
    hazard_viz = None
    hazard_labels = {}
    print(f'MGB landslide layer unavailable: {exc}')


soil_moisture_layer = None
try:
    soil_moisture_dataset = 'NASA_USDA/HSL/SMAP10KM_soil_moisture'
    soil_moisture_collection = (
        ee.ImageCollection(soil_moisture_dataset)
        .filterDate(start_date_recent, end_date_recent)
        .filterBounds(roi)
    )
    soil_moisture_count = soil_moisture_collection.size().getInfo() or 0
    if soil_moisture_count > 0:
        soil_moisture_layer = soil_moisture_collection.select('ssm').mean().clip(roi)
        print(f'Soil moisture images used: {soil_moisture_count}')
    else:
        print('No soil moisture data available for the selected period.')
except Exception as exc:
    soil_moisture_layer = None
    print(f'Soil moisture layer unavailable: {exc}')


High-risk threshold set at: 0.10
Using COPERNICUS/S2_SR_HARMONIZED for the recent period and COPERNICUS/S2_SR_HARMONIZED for the historical baseline.


C:\Users\LESTER\scoop\apps\anaconda3\current\App\Lib\site-packages\ee\deprecation.py:209: DeprecationWarning: 

Attention required for NASA_USDA/HSL/SMAP10KM_soil_moisture! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_SMAP10KM_soil_moisture

  warnings.warn(warning, category=DeprecationWarning)


No soil moisture data available for the selected period.


In [5]:
mean_ndvi_change_dict = ndvi_difference_masked.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=roi,
    scale=30,
)

mean_ndvi_change_value = None
mean_ndvi_change = mean_ndvi_change_dict.get('NDVI')
if mean_ndvi_change is not None:
    try:
        mean_ndvi_change_value = mean_ndvi_change.getInfo()
    except Exception as exc:
        print(f'Unable to retrieve NDVI change: {exc}')

if mean_ndvi_change_value is None:
    alert_message = (
        '>>> DATA WARNING: Unable to compute NDVI change for the selected period.\n'
        '>>> Please verify imagery availability or adjust the ROI and time range.\n'
    )
    print('Average NDVI change could not be determined.')
else:
    print(f'Average NDVI Change from 2020 to 2025: {mean_ndvi_change_value:.4f}')
    if mean_ndvi_change_value < VEGETATION_LOSS_THRESHOLD:
        alert_message = (
            '>>> RISK DETECTED: Significant vegetation loss identified.\n'
            '>>> This area is flagged as a high-risk hotspot.\n'
            '>>> Recommendation: Deploy Geo-Safe Sensor Nodes to this location.'
        )
    else:
        alert_message = (
            '>>> MONITORING: No significant large-scale vegetation loss detected.\n'
            '>>> Continue routine satellite monitoring of this area.'
        )

print('------------------------------------------------------------')
print(alert_message)
print('------------------------------------------------------------')


Average NDVI Change from 2020 to 2025: 0.1877
------------------------------------------------------------
>>> MONITORING: No significant large-scale vegetation loss detected.
>>> Continue routine satellite monitoring of this area.
------------------------------------------------------------


In [6]:
rgb_viz = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
ndvi_viz = {'min': -0.2, 'max': 1, 'palette': ['royalblue', 'white', 'green']}
change_viz = {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']}
soil_moisture_viz = {'min': 0.0, 'max': 0.6, 'palette': ['#f7fcf5', '#74c476', '#00441b']}
slope_viz = {'min': 0, 'max': 45, 'palette': ['#ffffcc', '#fd8d3c', '#800026']}
risk_viz = {'min': 0, 'max': 1, 'palette': ['#1a9641', '#ffffbf', '#d7191c']}
hotspot_viz = {'palette': ['#d7191c'], 'opacity': 0.6}

water_outline = water_mask_combined.updateMask(water_mask_combined).selfMask()
water_outline_style = {'palette': ['royalblue'], 'opacity': 0.4}

m = geemap.Map(center=[8.35, 124.65], zoom=12)
m.add_layer(historical_image.select(['B4', 'B3', 'B2']).clip(roi), rgb_viz, 'Historical Image (2020)')
m.add_layer(historical_ndvi_display.clip(roi), ndvi_viz, 'Historical NDVI (2020 Baseline)')
m.add_layer(recent_image.select(['B4', 'B3', 'B2']).clip(roi), rgb_viz, 'Recent Image (2025)')
m.add_layer(recent_ndvi_display.clip(roi), ndvi_viz, 'Recent NDVI (Veg & Water)')
m.add_layer(ndvi_difference_masked.clip(roi), change_viz, 'NDVI Change (Vegetation Only)')
m.add_layer(risk_score_masked.clip(roi), risk_viz, 'Composite Risk Score')
if soil_moisture_layer is not None:
    m.add_layer(soil_moisture_layer, soil_moisture_viz, 'Soil Moisture (SMAP)')
m.add_layer(slope.clip(roi), slope_viz, 'Slope (degrees)')
m.add_layer(risk_hotspots.clip(roi), hotspot_viz, 'High-Risk Hotspots')
m.add_layer(water_outline.clip(roi), water_outline_style, 'Water Mask')
m.addLayerControl()
m


Map(center=[8.35, 124.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [7]:
from pathlib import Path

output_dir = Path('outputs')
output_dir.mkdir(parents=True, exist_ok=True)

styled_risk_score = risk_score_masked.multiply(255).clamp(0, 255).toUint8().unmask(0)
styled_risk_hotspots = risk_hotspots.multiply(255).toUint8().unmask(0)
raw_risk_score = risk_score_masked.unmask(-9999)
raw_risk_hotspots = risk_hotspots.unmask(0)

try:
    geemap.ee_export_image(
        raw_risk_score,
        filename=str(output_dir / 'risk_score_raw.tif'),
        scale=30,
        region=roi,
        file_per_band=False,
    )
    geemap.ee_export_image(
        styled_risk_score,
        filename=str(output_dir / 'risk_score_visual.tif'),
        scale=30,
        region=roi,
        file_per_band=False,
    )
    geemap.ee_export_image(
        raw_risk_hotspots,
        filename=str(output_dir / 'risk_hotspots_raw.tif'),
        scale=30,
        region=roi,
        file_per_band=False,
    )
    geemap.ee_export_image(
        styled_risk_hotspots,
        filename=str(output_dir / 'risk_hotspots_visual.tif'),
        scale=30,
        region=roi,
        file_per_band=False,
    )
    print('Export complete: raw + visual GeoTIFFs saved in outputs/.')
except Exception as exc:
    print(f'Export skipped or failed: {exc}')


Generating URL ...
Please wait ...
Data downloaded to c:\Users\LESTER\Desktop\PhilSA Hackathon\outputs\risk_score_raw.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\LESTER\Desktop\PhilSA Hackathon\outputs\risk_score_visual.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\LESTER\Desktop\PhilSA Hackathon\outputs\risk_hotspots_raw.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\LESTER\Desktop\PhilSA Hackathon\outputs\risk_hotspots_visual.tif
Export complete: raw + visual GeoTIFFs saved in outputs/.
